In [89]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [93]:
%%cython --cplus -a

#distutils: define_macros=CYTHON_TRACE_NOGIL=1
#cython: boundscheck=False, wraparound=False

import numpy as np
import pandas as pd
import time


#target_mean_encoding实现
cpdef target_mean_v5(data, y_name, x_name):

    cdef int x_max = data.shape[0]          #dataframe的行数，即x或y列的记录数
        
    result = np.zeros(x_max,dtype=np.float64)
    cdef double[:] result_view = result       #使用memoryview优化  
    cdef double[:] value_arr = np.zeros(x_max)
    cdef double[:] count_arr = np.zeros(x_max)

    cdef long[:] x_values =data[x_name].values
    cdef long[:] y_values =data[y_name].values
    
    cdef int i = 0
    for i in range(x_max):
        value_arr[x_values[i]]+=y_values[i]
        count_arr[x_values[i]]+=1
    for i in range(x_max):
        
        result_view[i] = (value_arr[x_values[i]]-y_values[i])/(count_arr[x_values[i]]-1) 
    return result
        

    
  



In [101]:
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
target_mean_v3(data, 'y', 'x')

array([0.51639344, 0.52362205, 0.50869565, ..., 0.49486653, 0.52559055,
       0.51926606])

In [102]:
%%timeit 
target_mean_v5(data, 'y', 'x')

The slowest run took 6.16 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 32.7 µs per loop
